# Assignment 1: Uninformed Search

David Edwards

## Overview

The assignment was to create both breadth- and depth-first search algorithms in Python, implementing the algorithm detailed in [this notebook](http://nbviewer.jupyter.org/url/www.cs.colostate.edu/~anderson/cs440/notebooks/03%20Problem-Solving%20Agents.ipynb).

## Required Code

I have included the following three functions within this notebook:

  * `breadthFirstSearch(startState, goalState, successorsf)` 
  * `depthFirstSearch(startState, goalState, successorsf)`
  * `nodeSearch(startState, goalState, successorsf, breadthFirst)`
  
`breadthFirstSearch` and `depthFirstSearch` receive as arguments the starting state, the goal state, and a successors function.  `breadthFirstSearch` returns the breadth-first solution path as a list of states starting with the `startState` and ending with the `goalState`.  `depthFirstSearch` returns the depth-first solution path.

`nodeSearch` is used by both `breadthFirstSearch` and `depthFirstSearch` as the difference in implementation was only on one line.  `nodeSearch` takes in the same arguments as the other two search functions, as well as a boolean, `breadthFirst` which determines whether the search is breadth- or depth-first.


### Issues

I encountered a few issues while implementing this assignment.  
1. This was my first experience programming in Python, with the exception of modifying some Python 2 code for the [Department of Atmospheric Sciences](https://www.atmos.colostate.edu/).  That was trivial, as it was mostly moving paths, and adding parentheses to `print` statements.
2. My brain had a hard time remembering which nomenclature was used for Lists and Dictionaries.  
3. After being used to using a full-featured debugger, I was challenged by putting `print` statements throughout the code to try to figure out what was going on.  I ended up installing pyCharm, which helped immensely, once I figured out how to set it up for debugging.  Thankfully, I had used IntelliJ for CS455, so it wasn't too hard.
4. When adding the examples using the 10x10 graph and using Tuples, I was unable to replicate the given results.  That took me back to the drawing board, and back into pyCharm.  I noticed there, that I was duplicating Tuples, putting `(0,0)` into the children multiple times.  This seemed to be because the successors function returned not only what I considered to be successors, but also the initial child.  Once I removed that, it seemed to work much better.

In [3]:
def nodeSearch(startState, goalState, successorsf, breadthFirst):

    '''
    Given a startState, and goalState, and a function to
    calculate successors, and a boolean representing a breadthFirst search, 
    nodeSearch returns a breadthFirst search if breadthFirst==True, 
    or a depthFirst search if breadthFirst==False'''
    
    # Initialize expanded to be empty dictionary
    expanded = {}
    # Initialize unExpanded to be list containing (startState, None)
    unExpanded = [(startState, None)]

    if startState == goalState:
        return [startState]

    while unExpanded:

        state = unExpanded.pop()
        parent = state[0]

        # remove children which match parent
        children = [child for child in successorsf(parent) if child != parent  ]

        filteredChildren = []

        # Filter children, removing those in expanded, and whose key is in unExpanded, 
        # and where the child is the same as the parent
        for child in children:
            if child not in expanded and (child, parent) not in unExpanded and child != parent:
                filteredChildren.append(child)

        children = filteredChildren

        # don't overwrite the expanded dictionary if the parent is already in it
        if(parent not in expanded):
            expanded[parent] = state[1]

        # if the children contain the final state, iterate through the expanded
        # dictionary, adding parents to the solutionPath, then return it.
        if goalState in children:
            solutionPath = [state[0],goalState]

            parent = expanded[parent]
            while parent:
                solutionPath.insert(0, parent)
                parent = expanded[parent]

            return solutionPath

        # Sort and reverse the children
        children.sort()
        children.reverse()

        # convert the children to tuples of the child and parent
        childTuples = [(c, p) for c in children for p in [state[0]]]

        # if a Breadth-First search, add the childTuples to the front of the unExpanded list
        # if Depth-First, add the childTuples to the end of the unExpanded list
        if breadthFirst:
            unExpanded = childTuples + unExpanded
        else:
            unExpanded.extend(childTuples)
            
def breadthFirstSearch(startState, goalState, successorsf):
    '''
    Given a startState, and goalState, and a function to
    calculate successors, breadthFirstSearch returns a search of the nodes
    using a Breadth First Search algorithm '''
    
    return nodeSearch(startState, goalState, successorsf, True)


def depthFirstSearch(startState, goalState, successorsf):
    '''
    Given a startState, and goalState, and a function to
    calculate successors, depthFirstSearch returns a search of the nodes
    using a Depth First Search algorithm '''

    return nodeSearch(startState, goalState, successorsf, False)



## Examples

Here is a simple example.  States are defined by lower case letters.  A dictionary stores a list of successor states for each state in the graph that has successors.

In [4]:
successors = {'a':  ['b', 'c', 'd'],
              'b':  ['e', 'f', 'g'],
              'c':  ['a', 'h', 'i'],
              'd':  ['j', 'z'],
              'e':  ['k', 'l'],
              'g':  ['m'],
              'k':  ['z']}
import copy

def successorsf(state):
    return copy.copy(successors.get(state, []))

In [5]:
print('Breadth-first')
print('path from a to a is', breadthFirstSearch('a', 'a', successorsf))
print('path from a to m is', breadthFirstSearch('a', 'm', successorsf))
print('path from a to z is', breadthFirstSearch('a', 'z', successorsf))

Breadth-first
path from a to a is ['a']
path from a to m is ['a', 'b', 'g', 'm']
path from a to z is ['a', 'd', 'z']


In [6]:
print('Depth-first')
print('path from a to a is', depthFirstSearch('a', 'a', successorsf))
print('path from a to m is', depthFirstSearch('a', 'm', successorsf))
print('path from a to z is', depthFirstSearch('a', 'z', successorsf))

Depth-first
path from a to a is ['a']
path from a to m is ['a', 'b', 'g', 'm']
path from a to z is ['a', 'b', 'e', 'k', 'z']


Let's try a navigation problem around a grid of size 10 x 10.

In [7]:
def gridSuccessors(state):
    row, col = state
    # succs will be list of tuples () rather than list of lists [] because state must
    # be an immutable type to serve as a key in dictionary of expanded nodes
    succs = []
    for r in [-1, 0, 1]:
        for c in [-1, 0, 1]:
            newr = row + r
            newc = col + c
            if 0 <= newr <= 9 and 0 <= newc <= 9:  # cool, huh?
                succs.append( (newr, newc) )
    return succs

In [8]:
gridSuccessors([3,4])

[(2, 3), (2, 4), (2, 5), (3, 3), (3, 4), (3, 5), (4, 3), (4, 4), (4, 5)]

In [ ]:
gridSuccessors([3,9])

In [ ]:
gridSuccessors([0,0])

In [9]:
print('Breadth-first')
print('path from (0, 0) to (9, 9) is', breadthFirstSearch((0, 0), (8, 8), gridSuccessors))

Breadth-first
path from (0, 0) to (9, 9) is [(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8)]


In [10]:
print('Depth-first')
print('path from (0, 0) to (9, 9) is', depthFirstSearch((0, 0), (9, 9), gridSuccessors))

Depth-first
path from (0, 0) to (9, 9) is [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 8), (1, 7), (1, 6), (1, 5), (1, 4), (1, 3), (1, 2), (1, 1), (1, 0), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (1, 9), (2, 9), (3, 8), (3, 7), (3, 6), (3, 5), (3, 4), (3, 3), (3, 2), (3, 1), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (3, 9), (4, 9), (5, 8), (5, 7), (5, 6), (5, 5), (5, 4), (5, 3), (5, 2), (5, 1), (5, 0), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (5, 9), (6, 9), (7, 8), (7, 7), (7, 6), (7, 5), (7, 4), (7, 3), (7, 2), (7, 1), (7, 0), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 8), (9, 9)]


Oooo, what kind of path is that?  Let's plot it.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
path = depthFirstSearch((0, 0), (9, 9), gridSuccessors)
path

In [ ]:
rows = [location[0] for location in path]
cols = [location[1] for location in path]
plt.plot(rows,cols,'o-');

In [ ]:
path = breadthFirstSearch((0, 0), (9, 9), gridSuccessors)
path

In [ ]:
rows = [location[0] for location in path]
cols = [location[1] for location in path]
plt.plot(rows,cols,'o-');

In [ ]:
depthFirstSearch((0, 0), (9, 20), gridSuccessors)

## Grading

Your notebook will be run and graded automatically. Download [A1grader.tar](http://www.cs.colostate.edu/~anderson/cs440/notebooks/A1grader.tar)  and extract A1grader.py from it. Run the code in the following cell to demonstrate an example grading session. You should see a perfect score of 80/100 if your functions are defined correctly. 

The remaining 20% will be based on your writing.  In markdown cells, explain what your functions are doing and summarize the algorithms.

Add at least one markdown cell that describes problems you encountered in trying to solve this assignment.

In [ ]:
# Delete all variables defined so far (in notebook)
for name in dir():
    if not callable(globals()[name]) and not name.startswith('_'):
        del globals()[name]

# import numpy as np
import os
import copy

# import A1mysolution as mine
# import imp
# imp.reload(mine)
# breadthFirstSearch = mine.breadthFirstSearch
# depthFirstSearch = mine.depthFirstSearch

# trainValidateTestKFolds = mine.trainValidateTestKFolds
# trainLinear = mine.trainLinear
# evaluateLinear = mine.evaluateLinear
# trainNN = mine.trainNN
# evaluateNN = mine.evaluateNN

# import neuralnetworks as nn

# def within(correct, attempt, diff):
#     return np.abs((correct-attempt) / correct)  < diff

g = 0

for func in ['breadthFirstSearch', 'depthFirstSearch']:
    if func not in dir() or not callable(globals()[func]):
        print('CRITICAL ERROR: Function named \'{}\' is not defined'.format(func))
        print('  Check the spelling and capitalization of the function name.')

def chuck1():
    global g

    succs = {'a': ['b'], 'b':['c', 'd'], 'c':['e'], 'd':['f', 'i'], 'e':['g', 'h', 'i']}
    print('Searching this graph:\n', succs)
    def succsf(s):
        return copy.copy(succs.get(s,[]))

    print('Looking for path from a to b.')
    bfsCorrect = ['a', 'b']
    dfsCorrect = ['a', 'b']
    bfs = breadthFirstSearch('a', 'b', succsf)
    dfs = depthFirstSearch('a', 'b', succsf)
    
    if bfs == bfsCorrect:
        g += 20
        print('20/20 points. Your breadthFirstSearch found correct solution path of',bfs)
    else:
        print(' 0/20 points. Your breadthFirstSearch did not find correct solution path of',bfsCorrect)

    if dfs == dfsCorrect:
        g += 20
        print('20/20 points. Your depthFirstSearch found correct solution path of',dfs)
    else:
        print(' 0/20 points. Your depthFirstSearch did not find correct solution path of',dfsCorrect)

    print('Looking for path from a to i.')
    bfsCorrect = ['a', 'b', 'd', 'i']
    dfsCorrect = ['a', 'b', 'c', 'e', 'i']
    bfs = breadthFirstSearch('a', 'i', succsf)
    dfs = depthFirstSearch('a', 'i', succsf)
    if bfs == bfsCorrect:
        g += 20
        print('20/20 points. Your breadthFirstSearch found correct solution path of',bfs)
    else:
        print(' 0/20 points. Your breadthFirstSearch did not find correct solution path of',bfsCorrect)
    if dfs == dfsCorrect:
        g += 20
        print('20/20 points. Your depthFirstSearch found correct solution path of',dfs)
    else:
        print(' 0/20 points. Your depthFirstSearch did not find correct solution path of',dfsCorrect)

chuck1()

print('\n{} Grade is {}/100'.format(os.getcwd().split('/')[-1], g))
print('Up to 20 more points will be given based on the qualty of your descriptions of the method and the results.')


In [ ]:
%run -i A1grader.py